In [40]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [41]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
import random as python_random
import argparse
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle
import warnings
from tensorflow import keras
from keras.layers import LSTM, Activation, Dropout, Dense, Input, CuDNNLSTM
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import BatchNormalization
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score, classification_report
from keras_preprocessing.sequence import pad_sequences
import transformers
from transformers import (OpenAIGPTTokenizer, TFOpenAIGPTForSequenceClassification, MobileBertTokenizer,
                          TFMobileBertForSequenceClassification, TFAutoModelForSequenceClassification,
                          AutoTokenizer, BertTokenizerFast, TFBertForSequenceClassification,
                          DistilBertTokenizer, TFDistilBertForSequenceClassification,
                          RobertaTokenizer, TFRobertaForSequenceClassification,
                          XLNetTokenizer, TFXLNetForSequenceClassification)

warnings.filterwarnings("ignore")


In [43]:


def train_model(model, tokens_train, Y_train_bin, tokens_dev, Y_dev_bin,encoder,output_file):
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    model.fit(tokens_train, Y_train_bin, verbose=1, epochs=10,batch_size=64,callbacks=[callback], validation_data=(tokens_dev, Y_dev_bin))
    test_set_predict(model, tokens_dev, Y_dev_bin, "dev",encoder,output_file)
    return model

In [44]:
def test_set_predict(model,tokens_dev, Y_dev_bin, ident,encoder,output_file):
    '''Do predictions and measure accuracy on our own test set (that we split off train)'''
    # Get predictions using the trained model
    Y_pred = model.predict(tokens_dev)["logits"]
    Y_pred = Y_pred>0.5
    if output_file:
      pd.DataFrame(Y_pred).to_csv('/content/gdrive/MyDrive/Data/outputtest_dbert.csv')
    
    print(classification_report(Y_dev_bin, Y_pred,target_names= ["OFF",'NOT']))
    print('Accuracy on own {1} set: {0}'.format(round(accuracy_score(Y_dev_bin, Y_pred), 3), ident))

In [45]:
if __name__ == "__main__":
  train = pd.read_csv('/content/gdrive/MyDrive/Data/preprocessed data/processed_train.csv')
  val= pd.read_csv('/content/gdrive/MyDrive/Data/preprocessed data/processed_val.csv')
  test = pd.read_csv('/content/gdrive/MyDrive/Data/preprocessed data/processed_test.csv')
  X_train, Y_train = train['tweet'], train['task']
  X_dev, Y_dev = val['tweet'], val['task']
  X_test, Y_test = test['tweet'],test['task']

  encoder = LabelBinarizer()
  encoder = encoder.fit(Y_train.tolist())
  Y_train_bin = encoder.transform(Y_train.tolist())
  # Use encoder.classes_ to find mapping back
  Y_dev_bin = encoder.transform(Y_dev.tolist())      
  Y_test_bin = encoder.fit_transform(Y_test)
  filename = "/content/gdrive/MyDrive/Data/dbert"
  bert_pretrained= False
  custom_test_set = False
  output_file = False
  val_set = False
  if bert_pretrained:
    model = TFAutoModelForSequenceClassification.from_pretrained(filename, local_files_only=True)
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    tokenizer.pad_token = "[PAD]"
    tokens_test = tokenizer(X_test.values.tolist(), padding=True, max_length=100, truncation=True,
                           return_tensors="np").data
    test_set_predict(model, tokens_test, Y_test_bin, "test",encoder,output_file)
  else:
    # lm = "bert-base-uncased"
    # lm= "roberta-base"
    lm = "distilbert-base-uncased"
    optim = Adam(learning_rate=5e-5)
    loss_function = tf.keras.losses.BinaryCrossentropy(from_logits=True) 
    tokenizer = AutoTokenizer.from_pretrained(lm)
    model = TFAutoModelForSequenceClassification.from_pretrained(lm, num_labels=1)
    tokens_train = tokenizer(X_train.tolist(), padding=True, max_length=100,truncation=True, return_tensors="np").data
    tokens_dev = tokenizer(X_dev.tolist(), padding=True, max_length=100,truncation=True, return_tensors="np").data
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    model = train_model(model, tokens_train, Y_train_bin, tokens_dev, Y_dev_bin,output_file,encoder)
    test_file = True
    if test_file:
      tokens_test = tokenizer(X_test.tolist(), padding=True, max_length=100,truncation=True, return_tensors="np").data
      test_set_predict(model, tokens_test, Y_test_bin, "test",encoder,output_file)
    model.save_pretrained(filename)



Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_projector', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_897']
You should probably TRAIN this model on a down-stream task to be able to use 

Epoch 1/10
125/125 [==============================] - 86s 614ms/step - loss: 0.5386 - accuracy: 0.7000 - val_loss: 0.5141 - val_accuracy: 0.7918
Epoch 2/10
125/125 [==============================] - 75s 599ms/step - loss: 0.4018 - accuracy: 0.8184 - val_loss: 0.4674 - val_accuracy: 0.7978
Epoch 3/10
125/125 [==============================] - 75s 599ms/step - loss: 0.2761 - accuracy: 0.8830 - val_loss: 0.5521 - val_accuracy: 0.7718
Epoch 4/10
125/125 [==============================] - 75s 598ms/step - loss: 0.1573 - accuracy: 0.9438 - val_loss: 0.7762 - val_accuracy: 0.7618
Epoch 5/10
32/32 [==============================] - 5s 103ms/step
              precision    recall  f1-score   support

         OFF       0.84      0.81      0.82       647
         NOT       0.67      0.71      0.69       352

    accuracy                           0.78       999
   macro avg       0.75      0.76      0.76       999
weighted avg       0.78      0.78      0.78       999

Accuracy on own dev set: 0.